# Import Libraries

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
%matplotlib inline
sns.set_style("darkgrid")

from IPython.display import Image
from pydotplus import graph_from_dot_data
from sklearn import metrics

from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier

from sklearn.linear_model import LogisticRegression

from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve
from sklearn.metrics import auc
from sklearn.metrics import roc_auc_score

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split

from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_graphviz

from statsmodels.stats.outliers_influence import variance_inflation_factor

from xgboost import XGBClassifier

import joblib

pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 500)
plt.rcParams['figure.figsize'] = (30,30)

import warnings
warnings.filterwarnings("ignore")

# User Test Function

Run the following 2 codes to have a test on the functions created:

In [ ]:
from user_test import user_test

In [ ]:
user_test()

# Summary of Project 

## Data Collection & Transformation

## Clustering

## Classification

## Case Study

We will conduct the case study in 2 parts, the first part is the pre-match prediction and the second part is the post-match analysis.

__Djokovic beats Federer in Wimbledon 2019 Final over 5 sets in 4 hours and 57 minutes__

__Final Score: Djokovic 7-6(5) 1-6 7-6(4) 4-6 13-12(3) Federer__

--------------------------------------------------------------------------------------------------------------

The pre-match prediction model uses only 2 metrics: 
* __second_serves_return_points_won_%__
* __break_point_convert_%__

For this case study, we will use the following pre-match statistics available to us:
* __Tournament Average across first 6 matches before final__
* __YTD (2019) Player Statistics__
* __Previous H2H Accumulated Statistics prior to the match__

--------------------------------------------------------------------------------------------------------------

The match analysis model uses 6 metrics:
* __first_serves_points_won_%__
* __second_serves_points_won_%__
* __break_point_save_%__
* __first_serves_return_points_won_%__
* __break_point_convert_%__
* __break_points_return_total__

For this case study, we will use the following match statistics available to us:
* __Full Match Statistics__
* __Individual Sets Statistics__
* __Cumulative Win Probability over 5 sets__

### Extract Trained Models 

In [ ]:
# Load the trained models
prediction = joblib.load('prediction.pkl')  # for pre-match prediction

analysis = joblib.load('analysis.pkl')      # for match analysis

### Data Preparation

In [ ]:
# Load main dataset on matches
df_match = pd.read_csv('index_csv/clean_df_v3.0.csv')

In [ ]:
# Load case study file
df_wim_2019_final = df_match[df_match['match_id'] == '2019-540-MS001-10-1-d643-f324']

# Remove from matches data
df_match = df_match[df_match['match_id'] != '2019-540-MS001-10-1-d643-f324']
df_match.reset_index(drop = True, inplace = True)

In [ ]:
# Define features required for each analysis
pre_match = ['second_serves_return_points_won_%', 'break_point_convert_%']

match_analysis = ['first_serves_points_won_%', 'second_serves_points_won_%', 'break_point_save_%',
                  'first_serves_return_points_won_%', 'break_point_convert_%', 'break_points_return_total']

#### Tournament Performance

Here we wil extract the 6 matches for both players, Novak Djokovic and Roger Federer, leading up to the final. We will calculate the performance of the 2 features required by the model from the first 6 matches, which would account for their form going into the final.

In [ ]:
# Extract the 6 matches that both players played before final

df_2019 = df_match[df_match['year'] == 2019]
df_wim_2019 = df_2019[df_2019['tourney_slug'] == 'wimbledon']

df_djokovic = df_wim_2019[df_wim_2019['slug'] == 'd643']
df_federer = df_wim_2019[df_wim_2019['slug'] == 'f324']

In [ ]:
# Extract raw stats required to compute statistics

raw_stats = ['second_serve_return_won', 'second_serve_return_total',
            'break_points_converted', 'break_points_return_total']

df_djokovic = df_djokovic[raw_stats]
df_federer = df_federer[raw_stats]

In [ ]:
djokovic_1 = df_djokovic.sum()[0] / df_djokovic.sum()[1]
djokovic_2 = df_djokovic.sum()[2] / df_djokovic.sum()[3]

federer_1 = df_federer.sum()[0] / df_federer.sum()[1]
federer_2 = df_federer.sum()[2] / df_federer.sum()[3]

feature_1 = list([djokovic_1, federer_1])
feature_2 = list([djokovic_2, federer_2])

In [ ]:
tourney_average = pd.DataFrame(columns = pre_match)

tourney_average['second_serves_return_points_won_%'] = feature_1
tourney_average['break_point_convert_%'] = feature_2

#### YTD Player Statistics (01 Jan 2019 - 13 July 2019)

YTD Player Statistics from __1/1/2019 - 13/07/2019__

Wimbledon Final Date: __14/07/2019__

Extracted from: 
- Djokovic: https://www.ultimatetennisstatistics.com/playerProfile?playerId=4920
- Federer: https://www.ultimatetennisstatistics.com/playerProfile?playerId=3819

In [ ]:
feature_1 = [0.551, 0.503]
feature_2 = [0.50, 0.396]

ytd_average = pd.DataFrame(columns = pre_match)

ytd_average['second_serves_return_points_won_%'] = feature_1
ytd_average['break_point_convert_%'] = feature_2

#### Previous H2H Accumulated Statistics

This section aims to account for both players' performance when they face each other in previous head-to-head matches prior to the match. This dataframe would capture the match dynamic to be expected in this match.

In [ ]:
df_djokovic = df_match[df_match['slug'] == 'd643']
df_federer = df_match[df_match['slug'] == 'f324']

df_djokovic.reset_index(drop = True, inplace = True)
df_federer.reset_index(drop = True, inplace = True)

In [ ]:
# Filter only djokovic-federer match
for i in range(0,len(df_djokovic)):
    if 'f324' in df_djokovic['match_id'][i]:
        pass
    else:
        df_djokovic.drop(i, inplace = True)
        
# Filter only federer-djokovic match
for i in range(0,len(df_federer)):
    if 'd643' in df_federer['match_id'][i]:
        pass
    else:
        df_federer.drop(i, inplace = True)
        
# Remove the only other match they played in 2019
df_djokovic = df_djokovic[df_djokovic['match_id'] != '2019-605-MS016-1-10-f324-d643']
df_federer = df_federer[df_federer['match_id'] != '2019-605-MS016-1-10-f324-d643']

In [ ]:
# Extract raw stats required to compute statistics

raw_stats = ['second_serve_return_won', 'second_serve_return_total',
            'break_points_converted', 'break_points_return_total']

df_djokovic = df_djokovic[raw_stats]
df_federer = df_federer[raw_stats]

In [ ]:
djokovic_1 = df_djokovic.sum()[0] / df_djokovic.sum()[1]
djokovic_2 = df_djokovic.sum()[2] / df_djokovic.sum()[3]

federer_1 = df_federer.sum()[0] / df_federer.sum()[1]
federer_2 = df_federer.sum()[2] / df_federer.sum()[3]

feature_1 = list([djokovic_1, federer_1])
feature_2 = list([djokovic_2, federer_2])

In [ ]:
h2h_average = pd.DataFrame(columns = pre_match)

h2h_average['second_serves_return_points_won_%'] = feature_1
h2h_average['break_point_convert_%'] = feature_2

#### Full Match Statistics

This dataframe consists of 2 lines of data describing the player's individual full match statistics. Player 0 is Novak Djokovic and Player 1 is Roger Federer.

In [ ]:
df_full_match = df_wim_2019_final[match_analysis]

#### Individual Set Statistics

This dataset aims to capture the performance of both players by individual set during the match. Each set is represented as a row.

In [ ]:
df_djokovic_individual_stats = pd.DataFrame(columns = match_analysis)

df_djokovic_individual_stats['first_serves_points_won_%']        = [0.88, 0.27, 0.78, 0.80, 0.74]
df_djokovic_individual_stats['second_serves_points_won_%']       = [0.52, 0.33, 0.69, 0.30, 0.43]
df_djokovic_individual_stats['break_point_save_%']               = [1.00, 0.25, 1.00, 0.00, 0.60]
df_djokovic_individual_stats['first_serves_return_points_won_%'] = [0.25, 0.00, 0.13, 0.24, 0.27]
df_djokovic_individual_stats['break_point_convert_%']            = [0.00, 0.00, 0.00, 0.50, 0.33]
df_djokovic_individual_stats['break_points_return_total']        = [0.00, 0.00, 0.00, 2.00, 6.00]

df_federer_individual_stats = pd.DataFrame(columns = match_analysis)

df_federer_individual_stats['first_serves_points_won_%']        = [0.75, 1.00, 0.88, 0.76, 0.73]
df_federer_individual_stats['second_serves_points_won_%']       = [0.54, 0.33, 0.67, 0.54, 0.50]
df_federer_individual_stats['break_point_save_%']               = [0.00, 0.00, 0.00, 0.50, 0.67]
df_federer_individual_stats['first_serves_return_points_won_%'] = [0.12, 0.73, 0.22, 0.20, 0.26]
df_federer_individual_stats['break_point_convert_%']            = [0.00, 0.75, 0.00, 1.00, 0.40]
df_federer_individual_stats['break_points_return_total'] 

#### Cumulative Over Set Statistics

This section aims to capture the cumulative performance as the match progresses each set, each row will describe the cumulative statistics at the conclusion of each set.

In [ ]:
df_djokovic_cumulative_stats = pd.DataFrame(columns = match_analysis)

df_djokovic_cumulative_stats['first_serves_points_won_%']        = [0.88, 0.694, 0.73, 0.7435, 0.743]
df_djokovic_cumulative_stats['second_serves_points_won_%']       = [0.52, 0.466, 0.535, 0.4905, 0.47]
df_djokovic_cumulative_stats['break_point_save_%']               = [1.00, 0.4, 0.5, 0.375, 0.46]
df_djokovic_cumulative_stats['first_serves_return_points_won_%'] = [0.25, 0.189, 0.164, 0.179, 0.21]
df_djokovic_cumulative_stats['break_point_convert_%']            = [0.00, 0.00, 0.00, 0.50, 0.375]
df_djokovic_cumulative_stats['break_points_return_total']        = [0.00, 0.00, 0.00, 2.00, 8.00]

df_federer_cumulative_stats = pd.DataFrame(columns = match_analysis)

df_federer_cumulative_stats['first_serves_points_won_%']        = [0.75, 0.811, 0.836, 0.821, 0.79]
df_federer_cumulative_stats['second_serves_points_won_%']       = [0.54, 0.455, 0.516, 0.523, 0.51]
df_federer_cumulative_stats['break_point_save_%']               = [0.00, 0.00, 0.00, 0.50, 0.625]
df_federer_cumulative_stats['first_serves_return_points_won_%'] = [0.12, 0.306, 0.27, 0.256, 0.26]
df_federer_cumulative_stats['break_point_convert_%']            = [0.00, 0.6, 0.5, 0.625, 0.54]
df_federer_cumulative_stats['break_points_return_total']

### Case Study 

#### Prediction Model (XGBoost with Validation AUC: 76.73 %)

As requested, we have prepared 3 sets of data for this case study:
* <code> tourney_average </code> : This data describes both players' performance across first 6 matches leading up to the final
* <code> ytd_average     </code> : This data describes both players' performance from the year 2019 up until the final
* <code> h2h_average     </code> : This data describes both players' performance from all the past head-to-head matches


* Player 0 = Novak Djokovic
* Player 1 = Roger Federer

First we look at the prediction on the tourney average:

In [ ]:
prediction.predict(tourney_average)

In [ ]:
prediction.predict_proba(tourney_average)

As this is the accumulated statistics from their first 6 wins of the tournament, the .predict method becomes trivial as they are obviously expected to win with the statistics collected from their wins. 

However, more information can be obtained from the .predict_proba method as we can see who had played better during their first 6 matches before the final. As shown above, we can see Federer being more dominant in his wins, with his statistics describing a __77.31 %__ match win rate compared to Djokovic's __71.14 %__.

This becomes more impressive when taken into account that Federer had also face harder opponents on his run to the final compared to Djokovic.

__Paths To Final:__

| Rounds |       Novak Djokovic       |      Roger Federer     |
|:------:|:--------------------------:|:----------------------:|
|   R1   |    Philipp Kohlschreiber   |      Lloyd Harris      |
|   R2   |         Denis Kudla        |       Jay Clarke       |
|   R3   |       Hubert Hurkacz       |   Lucas Pouille [27]   |
|   R4   |         Ugo Humbert        | Matteo Berrettini [17] |
|   QF   |      David Goffin [21]     |    Kei Nishikori [8]   |
|   SF   | Roberto Bautista-Agut [23] |    Rafael Nadal [3]    |

Federer had faced 4 seeded players and 2 top 10 players (including Rafael Nadal) while Djokovic had not faced any top 20 players before the final. 

Using this dataset as a prediction for the match would give Federer an edge of 2 to 4 % against Djokovic.

Next, we look at the prediction based on their Year-to-Date statistics prior to this final.

In [ ]:
prediction.predict(ytd_average)

In [ ]:
prediction.predict_proba(ytd_average)

Prior to this match, Federer had accumulated 32 Wins and 4 Losses (88.89 %) in 2019 while Djokovic had accumulated 28 Wins and 6 Losses (82.35 %). With this information, the .predict method once again becomes trivial as they both had a successful season at 80 + % win rate. 

However, we can see some interesting figures when we look at the .predict_proba method. Though Federer had a more successful season with higher win rate, Djokovic actually had been more dominant with his game on-court throughout the year, his average match stats gives him a __67.03 %__ of winning any match while Federer had __55.12 %__, indicating less dominant wins in the year compared to Djokovic.

Based on this dataset, we would expect Djokovic to win the final with an edge of 5-10 % over Federer.

Finally, we look at their previous head-to-head records to better capture their contrasting playing style and the match dynamic when the two of them face each other.

In [ ]:
prediction.predict(h2h_average)

In [ ]:
prediction.predict_proba(h2h_average)

Before this match, Djokovic held a 25-22 winning record over Federer.

With the .predict method, we can see that both players bring a good level of tennis when they face each other, as both compiled statistics are capable of winning matches based on the model. 

Next, we look at the .predict_proba results, we can see that Federer stats had been more dominant than Djokovic in their past matches, at __60.23 %__ against Djokovic's __51.04 %__. This is a relative dominance ratio of 54:46 in favor to Federer. This is an interesting statistic as Djokovic held a 25-22 winning head-to-head against Federer. 

*This may be due to the matches earlier in their career when Federer was at his prime while Djokovic is still struggling to challenge Federer or Nadal.*

Using this model, Federer would be expected to win the final with an edge of 5-10%. However, this is the least accurate model of the 3 as we are taking into account matches from 2006 - 2018 between them, with no additional bias given to the recent matches, as there had been a dynamic switch during the time period.

#### Analysis Model (AdaBoost with Validation AUC: 92.60 %)

As requested, we have prepared 3 sets of data for this case study:
* <code> df_full_match                </code> : This data describes both players' performance during the final
* <code> df_djokovic_individual_stats </code> : This data describes djokovic's performance in each of the 5 sets played
* <code> df_federer_individual_stats </code>  : This data describes federer's performance in each of the 5 sets played
* <code> df_djokovic_cumulative_stats </code> : This data describes djokovic's performance cumulatively at end of each set
* <code> df_federer_cumulative_stats </code>  : This data describes federer's performance cumulatively at end of each set


* Player 0 = Novak Djokovic
* Player 1 = Roger Federer

First, we look at the full match stats.

In [ ]:
analysis.predict(df_full_match)

In [ ]:
analysis.predict_proba(df_full_match)

The analysis model that we built predicted a win for Federer based on the full match stats, which perhaps deservedly so as Federer was the better player throughout the whole match, won 14 more points than Djokovic and had 2 match points on his own serve but failed to convert in the final set. This would be a false prediction by the analysis model but it does give insights into the relative performance and which player's performance is more deserving for victory. 

When we look at the win probability for each player, we can see the model predicted a 49.42 % chance of winning for Djokovic and a 51.56 % chance of winning for Federer, which translates to a relative win rate of 51 % to 49 % in favor of Federer. Therefore, Federer did play better than Djokovic and would have probably won the match any other day, but the model shows that Djokovic did not play terrible but was about 4 % below Federer's level on the day. The difference was Djokovic increased his level when it comes to pressure points like facing Federer's break points and match points opportunities. This helped him stay in the match until the 5th set despite playing at a lower level than Federer, we will look into a more detailed set-by-set analysis now.

__Final Score:__
- __Set 1: Djokovic won 7-6(5)__
- __Set 2: Federer won 6-1__
- __Set 3: Djokovic won 7-6(4)__
- __Set 4: Federer won 6-4__
- __Set 5: Djokovic won 13-12(3)__

In [ ]:
# Individual Set Outcome Prediction
print(f"Djokovic prediction on each set:{analysis.predict(df_djokovic_individual_stats)}")
print(f"Federer prediction on each set: {analysis.predict(df_federer_individual_stats)}")
print("")

# Chance of Winning in Eact Set by Djokovic
print("Djokovic Chance of Winning in Each Set")
for i in range(0, len(analysis.predict_proba(df_djokovic_individual_stats))):
    print(f"Set {str(1 + i)}: {round(analysis.predict_proba(df_djokovic_individual_stats)[i][1] * 100, 2)} %")
    
print("")

# Chance of Winning in Eact Set by Federer
print("Federer Chance of Winning in Each Set")
for i in range(0, len(analysis.predict_proba(df_federer_individual_stats))):
    print(f"Set {str(1 + i)}: {round(analysis.predict_proba(df_federer_individual_stats)[i][1] * 100, 2)} %")

print("")

# Break Point Chances by Both Players
print (f"Number of Break Point Opportunities:\n")

for i in range(0, len(df_djokovic_individual_stats['break_points_return_total'])):
    print (f"Set {1 + i}: Djokovic   {df_djokovic_individual_stats['break_points_return_total'][i]}      Federer   {df_federer_individual_stats['break_points_return_total'][i]}")

print("-----------------------------------------")
print(f"Total: Djokovic   {df_djokovic_individual_stats['break_points_return_total'].sum()}      Federer  {df_federer_individual_stats['break_points_return_total'].sum()}")

As we conducted an initial analysis on ~100k matches on Section 3 of this document, no player had ever won a match without breaking opponent's serve from 1990-2019. It is theoretically possible but highly improbable as the statistics shows, but assuming they played a normal Best-of-3 Sets match, Djokovic would have been the only player in 2 decades to accomplish that feat. Djokovic had 0 break point opportunities in the first 3 sets as compared to Federer's 6, but ended the set 3 with a 2-1 lead in sets, winning the first and third set via tiebreakers.

As we can see from the predict function above, based on their performance, Federer was forecasted to win the first 3 sets while Djokovic was forecasted to lose all 3 sets. Most tennis speculators think Federer lost the match in the 5th set when he led 8:7 in games and 40:15 in points with 2 match points on his serve, but in reality Federer lost his match in 1st and 3rd set where he played significantly better than Djokovic in the 2 sets but failed to convert his break points to capitalize on his superior performance over djokovic, and partially due to Djokovic stepping up his game in crucial moments to win all 3 tiebreakers that led him to the victory.

When we look at the win probability of each player in each set, we can see fluctuating performance from Djokovic and a consistent performance from Federer before falling in set 4 and 5. This is not surprising as Federer was turning 38 years old in a month during that match and Djokovic was 6 years younger than him, thus giving Djokovic an advantage as the matches went over 3-4 hours as he had better physical endurance.

When we look at the final set, we can see that the model predicted both players to lose, with a probability of winning at 49.04 % for Djokovic and 49.64 % for Federer, signifying that both player's performance are not worthy of winning, but it also shows that both players are playing at a similar level, hence the game went on to the 12-12 in games, requiring a final set tiebreaker to decide the winner. 

Though they played at a similar level in the final set, Djokovic's final set performance was his personal 2nd best set in terms of level of play, while Federer's final set performance was his personal 2nd worst set, indicating Djokovic improved his performance after set 3 and Federer's level falling gradually after set 3. We will try to investigate this further in our next analysis, which looks at the cumulative performance at the conclusion of each set.

In [ ]:
# Calculating relative performance between 2 players

djo_cumulative_performance = []
fed_cumulative_performance = []
for i in range(0, len(analysis.predict_proba(df_djokovic_cumulative_stats))):
    djo_cumulative_performance.append(analysis.predict_proba(df_djokovic_cumulative_stats)[i][1])
    fed_cumulative_performance.append(analysis.predict_proba(df_federer_cumulative_stats)[i][1])

djo_relative_performance = [0.5]
fed_relative_performance = [0.5]
for i in range(0, len(djo_cumulative_performance)):
    djo_relative_performance.append(round(djo_cumulative_performance[i] / (djo_cumulative_performance[i] + fed_cumulative_performance[i]), 2))
    fed_relative_performance.append(round(fed_cumulative_performance[i] / (djo_cumulative_performance[i] + fed_cumulative_performance[i]), 2))


# Plot 2 lines displaying momentum and performance change during the 5 sets
sns.lineplot(x=range(0,6), y=djo_relative_performance)
sns.lineplot(x=range(0,6), y=fed_relative_performance)
plt.rcParams.update({'font.size': 20})
plt.rcParams['figure.figsize'] = (18,18)
plt.legend(['Djokovic', 'Federer'])
plt.title("Momentum Shift in Wimbledon 2019 Final")
plt.xlabel("Sets")
plt.ylabel("Relative Performance")

Here, we can see the shift in their relative performance from start of the match to the end, showing that Federer gradually increase his dominance over Djokovic from the first point to the end of set 2, with the peak at 57% - 43 % in relative performance in favor of Federer, a staggering 14 % edge over Djokovic. 

After set 2, Federer's relative performance against Djokovic dipped slightly in set 3 and continued to drop further in set 4 and 5, but statistically remain the superior player throughout the whole match from first point to the last point.

#### Improvements

In [ ]:
# Load djokovic full match stats
df_djo_improve = df_full_match.drop(85168)

# Improvement rate
rate = 1.1
baseline = round(analysis.predict_proba(df_djo_improve)[0][1] * 100, 2)
print(f"Original chance of winning full match: {baseline} %\n")

# Analyze which feature most beneficial for increasing chance of winning
for i in df_djo_improve.columns:
    df_djo_improve = df_full_match.drop(85168)    # Reset values for comparison
    df_djo_improve[i] = df_djo_improve[i] * rate
    prediction = round(analysis.predict_proba(df_djo_improve)[0][1] * 100, 2)
    print(f"{i}: Chance of winning improved by {round((prediction - baseline), 2)} %")

As we can see from the output, Djokovic would benefit most from improving the first serves points won %. By improving that feature alone by 10 %, Djokovic would gain 0.55 % more chance of winning the match. If all 6 features were improved by 10 %, Djokovic would gain 1.43 % more chance of winning the match.

In [ ]:
# Load federer full match stats
df_fed_improve = df_full_match.drop(64598)

# Improvement rate
rate = 1.1
baseline = round(analysis.predict_proba(df_fed_improve)[0][1] * 100, 2)
print(f"Original chance of winning full match: {baseline} %\n")

# Analyze which feature most beneficial for increasing chance of winning
for i in df_fed_improve.columns:
    df_fed_improve = df_full_match.drop(64598)    # Reset values for comparison
    df_fed_improve[i] = df_fed_improve[i] * rate
    prediction = round(analysis.predict_proba(df_fed_improve)[0][1] * 100, 2)
    print(f"{i}: Chance of winning improved by {round((prediction - baseline), 2)} %")

Likewise, Federer would benefit most by improving the first serves points won %. By improving that feature alone by 10 %, Federer would gain 0.53 % more chance of winning the match. If all 6 features were improved by 10 %, Federer would gain 1.31 % more chance of winning the match.

### Summary & Recommendation

__Facts & Findings:__
1. Final Score: Djokovic 7-6(5) 1-6 7-6(4) 4-6 13-12(3) Federer
2. Federer performed better in every statistics overall:
    * Won 14 more total points (218 - 204)
    * Higher Service Points Won % (68 % - 64 %)
    * Higher Return Points Won % (36 % - 32 %)
    * 5 more break point opportunities (13 - 8)
    * Converted 4 more break points (7 - 3)
    * Higher break point conversion rate (54 % - 38 %)
    * Higher break point save rate (63 % - 46 %) 
3. Djokovic led by 2 sets to 1 without having a break point opportunity
    * Theoretically won a BO3 match without breaking opponent's serve (Never happened in a single match from 1990 - 2019)    
4. Djokovic won the match despite winning 14 less points in total
    * Total % of matches where winners won with a point deficit: 4.58 %    
5. Pre-match prediction model results:
    * **Tournament Average: Federer with a 52.08 % chance of winning**
    * **Year-To-Date      : Djokovic with a 54.87 % chance of winning**
    * **Head-to-Head      : Federer with a 54.13 % chance of winning**
    - *Head-to-head model not as reliable as tournament average and year-to-date data as it consists of data from 2006 - 2018 with no recency bias*
6. Post-match analysis model results:
    * **Set 1     : Federer with a 53.22 % chance of winning**
    * **Set 2     : Federer with a 59.06 % chance of winning**
    * **Set 3     : Federer with a 53.88 % chance of winning**
    * **Set 4     : Djokovic with a 52.76 % chance of winning**
    * **Set 5     : Federer with a 50.30 % chance of winning**
    * **Full Match: Federer with a 51.06 % chance of winning**
7. Recommendation For Improvements:
    * **Based on the model, player would benefit most by improving win rate on the first serves point**